<a href="https://colab.research.google.com/github/moemag/PCA-LDA-Implementation/blob/main/PCA%20%26%20LDA%20implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PCA Implementation


In [ ]:
from PIL import Image
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier


x = []
y = []
count = 1

for i in range(1,41):
  images = os.listdir('./drive/MyDrive/UNI/att-database-of-faces/s'+str(i))
  count = 1
  for image in images:
    image_temp = np.array(Image.open('./drive/MyDrive/UNI/att-database-of-faces/s'+str(i) +'/'+str(count)+'.pgm')).flatten()
    x.append(image_temp)
    y.append(str(i))
    count += 1

data = np.array(x)
data_ids = np.array(y)


training = np.empty((0,10304), int)
training_ids = np.array([])

testing = np.empty((0,10304), int)
testing_ids = np.array([])

for i in range(400):
  if i%2 != 0:
    training = np.vstack([training, data[i]])
    training_ids = np.append(training_ids, data_ids[i])
  else:
    testing = np.vstack([testing, data[i]])
    testing_ids = np.append(testing_ids, data_ids[i])



data_mean =  np.mean(data, axis=0)
data_z = data - data_mean

data_cov = np.dot(data_z.T, data_z) / len(data)
eigen_values, eigen_vectors = np.linalg.eigh(data_cov)

eigen_vectors = eigen_vectors.T

i = eigen_values.argsort()[::-1]
eigen_values = eigen_values[i]
eigen_vectors = eigen_vectors[:, i]

sum = np.sum(eigen_values)

alpha = np.array([0.8, 0.85, 0.9, 0.95])
fr = 0
r = [0, 0, 0, 0]

#for i in range(4):
#    temp_sum = 0
#    while fr < alpha[i]:
#        temp_sum += eigen_values[r[i]]
#        fr = temp_sum / sum
#        r[i] += 1

for i in range(4):
  temp_sum = 0
  while fr < alpha[i]:
    temp_sum += eigen_values[r[i]]
    fr = temp_sum / sum
    r[i] += 1



  projection_matrix = eigen_vectors[:, 0:r[i]]

  training_mean = np.mean(training)
  centered_training = training -  training_mean
  projected_training = np.dot(centered_training, projection_matrix)

  testing_mean = np.mean(testing)
  centered_testing = testing - testing_mean
  projected_testing = np.dot(centered_testing, projection_matrix)

  model = KNeighborsClassifier(n_neighbors=1)
  model.fit(projected_training,training_ids)
  predicted= model.predict(projected_testing)

  accuracy = (np.sum(testing_ids == predicted) / len(testing_ids)) * 100
  print("Accuracy for alpha = ", alpha[i], "is ", str(accuracy), "%")



Accuracy for alpha =  0.8 is  88.0 %
Accuracy for alpha =  0.85 is  92.0 %
Accuracy for alpha =  0.9 is  93.0 %
Accuracy for alpha =  0.95 is  93.0 %


LDA Implementation


In [ ]:
# This is a sample Python script.
from PIL import Image
import numpy as np
from numpy import linalg as la

x = []
y = []
count = 1

for i in range(1,41):
  images = os.listdir('./drive/MyDrive/UNI/att-database-of-faces/s'+str(i))
  count = 1
  for image in images:
    image_temp = np.array(Image.open('./drive/MyDrive/UNI/att-database-of-faces/s'+str(i) +'/'+str(count)+'.pgm')).flatten()
    x.append(image_temp)
    y.append(str(i))
    count += 1

data = np.array(x)
data_ids = np.array(y)


training = np.empty((0,10304), int)
training_ids = np.array([])

testing = np.empty((0,10304), int)
testing_ids = np.array([])

for i in range(400):
  if i%2 != 0:
    training = np.vstack([training, data[i]])
    training_ids = np.append(training_ids, data_ids[i])
  else:
    testing = np.vstack([testing, data[i]])
    testing_ids = np.append(testing_ids, data_ids[i])





data_mean =  np.mean(data, axis=0)
data_z = data - data_mean

data_cov = np.dot(data_z.T, data_z) / len(data)

class_mean = np.empty((0, 10304))
mean_vec = []
sum = 0
for i in range(40):
    for j in range(10304):
        for k in range(10):
            idx = 10*i+k
            sum += data[idx][j]
        mean_vec.append(sum/10)
        sum = 0
    class_mean = np.vstack([class_mean, mean_vec])
    mean_vec = []

#training_mean = np.mean(training, axis=0)
diff = np.array(class_mean[0] - data_mean)
Sb = np.dot(diff[:, None], diff[None, :])
for i in range(1, 40):
    diff = np.array(class_mean[i] - data_mean)
    Sb += np.dot(diff[:, None], diff[None, :])
Sb *= 10


S = np.zeros((10304, 10304))
Zi = np.empty((0, 10304))
for j in range(400):
    z = data[j] - class_mean[int(j/10)]
    Zi = np.vstack([Zi, z])
    if (j+1) % 10 == 0:
        S += np.matmul(Zi.T, Zi)
        Zi = np.empty((0, 10304))


S_inv = la.inv(S)
evals, evecs = np.linalg.eigh(np.matmul(S_inv, Sb))

evecs = evecs.T

i = evals.argsort()[::-1]
evals = evals[i]
evecs = evecs[:, i]

projection_matrix = evecs[:, 0:39]

print(len(projection_matrix))
print(len(projection_matrix[0]))

training_mean = np.mean(training)
centered_training = training -  training_mean
projected_training = np.dot(centered_training, projection_matrix)


testing_mean = np.mean(testing)
centered_testing = testing - testing_mean
projected_testing = np.dot(centered_testing, projection_matrix)


model = KNeighborsClassifier(n_neighbors=1)
model.fit(projected_training,training_ids)
predicted= model.predict(projected_testing)

accuracy = (np.sum(testing_ids == predicted) / len(testing_ids)) * 100
print("Accuracy =" , str(accuracy), "%")

10304
39
Accuracy = 86.0 %
